In [1]:

    !pip install transformers
    !pip install gradio
    !pip install timm
    !pip install inflect
    !pip install phonemizer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.4/314.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.1 MB/s et

In [2]:
    !sudo apt-get update
    !sudo apt-get install espeak-ng
    !pip install py-espeak-ng

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [825 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,756 kB]
Get:13 http://archive.ubuntu.com/ubuntu

In [3]:

import io
import matplotlib.pyplot as plt
import requests
import inflect
from PIL import Image


In [4]:
def load_image_from_url(url):
    return Image.open(requests.get(url, stream=True).raw)

In [5]:
def render_results_in_image(in_pil_img, in_results):
    plt.figure(figsize=(16, 10))
    plt.imshow(in_pil_img)

    ax = plt.gca()

    for prediction in in_results:

        x, y = prediction['box']['xmin'], prediction['box']['ymin']
        w = prediction['box']['xmax'] - prediction['box']['xmin']
        h = prediction['box']['ymax'] - prediction['box']['ymin']

        ax.add_patch(plt.Rectangle((x, y),
                                   w,
                                   h,
                                   fill=False,
                                   color="green",
                                   linewidth=2))
        ax.text(
           x,
           y,
           f"{prediction['label']}: {round(prediction['score']*100, 1)}%",
           color='red'
        )

    plt.axis("off")

    # Save the modified image to a BytesIO object
    img_buf = io.BytesIO()
    plt.savefig(img_buf, format='png',
                bbox_inches='tight',
                pad_inches=0)
    img_buf.seek(0)
    modified_image = Image.open(img_buf)

    # Close the plot to prevent it from being displayed
    plt.close()

    return modified_image

In [6]:
def summarize_predictions_natural_language(predictions):
    summary = {}
    p = inflect.engine()

    for prediction in predictions:
        label = prediction['label']
        if label in summary:
            summary[label] += 1
        else:
            summary[label] = 1

    result_string = "In this image, there are "
    for i, (label, count) in enumerate(summary.items()):
        count_string = p.number_to_words(count)
        result_string += f"{count_string} {label}"
        if count > 1:
          result_string += "s"

        result_string += " "

        if i == len(summary) - 2:
          result_string += "and "

    # Remove the trailing comma and space
    result_string = result_string.rstrip(', ') + "."

    return result_string

In [9]:
from transformers import pipeline

In [10]:
model = pipeline("object-detection","facebook/detr-resnet-50")

config.json:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

In [11]:
from PIL import Image

In [30]:
raw_image = Image.open('/content/image.jpg')
raw_image.resize((569, 491))

UnidentifiedImageError: cannot identify image file '/content/image.jpg'

In [17]:
import os
os.getcwd()

'/content'

In [21]:
output  = model("https://img.freepik.com/free-vector/sticker-set-mixed-daily-objects_1308-108542.jpg?size=626&ext=jpg&ga=GA1.1.1224184972.1714435200&semt=ais")

In [23]:
output

[{'score': 0.9752901196479797,
  'label': 'car',
  'box': {'xmin': 88, 'ymin': 366, 'xmax': 230, 'ymax': 457}},
 {'score': 0.9959814548492432,
  'label': 'scissors',
  'box': {'xmin': 54, 'ymin': 245, 'xmax': 189, 'ymax': 340}}]

In [29]:
output  = model("https://c.ndtvimg.com/2022-06/fdp0lr_car_625x300_17_June_22.jpg")
output

[{'score': 0.9935601949691772,
  'label': 'person',
  'box': {'xmin': 438, 'ymin': 303, 'xmax': 513, 'ymax': 427}},
 {'score': 0.9935994148254395,
  'label': 'motorcycle',
  'box': {'xmin': 442, 'ymin': 346, 'xmax': 518, 'ymax': 463}},
 {'score': 0.9994643330574036,
  'label': 'motorcycle',
  'box': {'xmin': 628, 'ymin': 321, 'xmax': 800, 'ymax': 560}},
 {'score': 0.9963983297348022,
  'label': 'car',
  'box': {'xmin': 292, 'ymin': 283, 'xmax': 365, 'ymax': 341}},
 {'score': 0.9761282205581665,
  'label': 'car',
  'box': {'xmin': 265, 'ymin': 256, 'xmax': 310, 'ymax': 285}},
 {'score': 0.9198486804962158,
  'label': 'person',
  'box': {'xmin': 437, 'ymin': 304, 'xmax': 502, 'ymax': 426}},
 {'score': 0.9666690230369568,
  'label': 'motorcycle',
  'box': {'xmin': 501, 'ymin': 342, 'xmax': 554, 'ymax': 450}},
 {'score': 0.949428141117096,
  'label': 'car',
  'box': {'xmin': 266, 'ymin': 245, 'xmax': 310, 'ymax': 277}},
 {'score': 0.9986584186553955,
  'label': 'car',
  'box': {'xmin': 100